In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

# custom module imports
# import acquire as aq
import Prepare as pr
# import explore as ex

# feature selection imports
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE

# import scaling methods
from sklearn.preprocessing import RobustScaler, StandardScaler
from scipy import stats
from sklearn.model_selection import train_test_split

# import modeling methods
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import explained_variance_score
from scipy import stats

# import to remove warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('salaries.csv')

In [3]:
df.head()

,AGY,NAME,LASTNAME,FIRSTNAME,MI,JOBCLASS,JC TITLE,RACE,SEX,EMPTYPE,...,RATE,HRSWKD,MONTHLY,ANNUAL,STATENUM,duplicated,multiple_full_time_jobs,combined_multiple_jobs,summed_annual_salary,hide_from_search
0,101,SENATE ...,GILLIAM,STACEY,L,7101,LEG. OFFICIAL/ADMINISTRATOR ...,WHITE,FEMALE,URP - UNCLASSIFIED REGULAR PART-TIME,...,0.0,20.0,8100.0,97200.0,339371,True,NaN,NaN,181200.0,NaN
1,104,LEGISLATIVE BUDGET BOARD ...,GILLIAM,STACEY,L,C160,COMMITTEE DIRECTOR ...,WHITE,FEMALE,URP - UNCLASSIFIED REGULAR PART-TIME,...,0.0,20.0,7000.0,84000.0,339371,True,NaN,NaN,NaN,True
2,101,SENATE ...,NELSON,DAVID,,7101,LEG. OFFICIAL/ADMINISTRATOR ...,WHITE,MALE,URP - UNCLASSIFIED REGULAR PART-TIME,...,0.0,20.0,9500.0,114000.0,193187,True,NaN,NaN,210000.0,NaN
3,104,LEGISLATIVE BUDGET BOARD ...,NELSON,DAVID,,P080,SENIOR BUDGET ADVISOR ...,WHITE,MALE,URP - UNCLASSIFIED REGULAR PART-TIME,...,0.0,20.0,8000.0,96000.0,193187,True,NaN,NaN,NaN,True
4,101,SENATE ...,ROCHA,MARIE,S,7103,LEG. SERVICE/MAINTENANCE ...,HISPANIC,FEMALE,URF - UNCLASSIFIED REGULAR FULL-TIME,...,0.0,41.0,3365.4,40384.8,152257,True,NaN,True,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144738 entries, 0 to 144737
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   AGY                      144738 non-null  int64  
 1   NAME                     144738 non-null  object 
 2   LASTNAME                 144738 non-null  object 
 3   FIRSTNAME                144738 non-null  object 
 4   MI                       144738 non-null  object 
 5   JOBCLASS                 144738 non-null  object 
 6   JC TITLE                 144738 non-null  object 
 7   RACE                     144738 non-null  object 
 8   SEX                      144738 non-null  object 
 9   EMPTYPE                  144738 non-null  object 
 10  HIREDT                   144738 non-null  object 
 11  RATE                     144738 non-null  float64
 12  HRSWKD                   144738 non-null  float64
 13  MONTHLY                  144738 non-null  float64
 14  ANNU

In [5]:
df.RACE.value_counts()

WHITE              64651
HISPANIC           39557
BLACK              33964
ASIAN               4419
OTHER               1453
AM INDIAN            694
Name: RACE, dtype: int64

In [6]:
df['JC TITLE'].value_counts()

CORREC  OFFICER IV                                    8753
CORREC  OFFICER V                                     7350
TEXAS WORKS ADVISOR II                                3812
CORREC OFFCR III                                      3652
DIRECT SUPPORT PROFESSIONAL I                         2484
                                                      ... 
INVESTIGATIVE ANALYST                                    1
EXECUTIVE DIRECTOR FOR ERS                               1
MILITARY SPECIALIST V                                    1
EXECUTIVE DIRECTOR/SECRETARY                             1
ADMINISTRATIVE ASSISTANT IV                              1
Name: JC TITLE, Length: 1406, dtype: int64

In [5]:
df = pr.prepare_tex(df)

In [6]:
df = pr.create_features(df)

In [7]:
train, validate, test = pr.split_data(df)

Making Sure Our Shapes Look Good
Train: (81048, 20), Validate: (34736, 20), Test: (28946, 20)


In [8]:
train.columns

Index(['agency_id', 'agency', 'lastname', 'firstname', 'title', 'race', 'sex',
       'emptype', 'hire_date', 'hours_worked', 'monthly_salary',
       'annual_salary', 'is_female', 'is_white', 'is_hispanic', 'is_black',
       'is_BIPOC', 'race_encoded', 'tenure_months', 'tenure_years'],
      dtype='object')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81048 entries, 125096 to 89798
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   agency_id       81048 non-null  int64         
 1   agency          81048 non-null  object        
 2   lastname        81048 non-null  object        
 3   firstname       81048 non-null  object        
 4   title           81048 non-null  object        
 5   race            81048 non-null  object        
 6   sex             81048 non-null  object        
 7   emptype         81048 non-null  object        
 8   hire_date       81048 non-null  datetime64[ns]
 9   hours_worked    81048 non-null  float64       
 10  monthly_salary  81048 non-null  float64       
 11  annual_salary   81048 non-null  float64       
 12  is_female       81048 non-null  int64         
 13  is_white        81048 non-null  int64         
 14  is_hispanic     81048 non-null  int64         
 1

In [10]:
# assign X and y variables in all our split dfs
X_train = train[['agency_id', 'hours_worked','is_female', 'is_white', 
                'is_hispanic', 'is_black', 'is_BIPOC', 'race_encoded', 'tenure_months',
                'tenure_years']]
y_train = train[['annual_salary']]

X_validate = validate[['agency_id', 'hours_worked','is_female', 'is_white', 
                'is_hispanic', 'is_black', 'is_BIPOC', 'race_encoded', 'tenure_months',
                'tenure_years']]
y_validate = validate[['annual_salary']]

X_test = test[['agency_id', 'hours_worked','is_female', 'is_white', 
                'is_hispanic', 'is_black', 'is_BIPOC', 'race_encoded', 'tenure_months',
                'tenure_years']]
y_test = test[['annual_salary']]

In [11]:
def select_rfe(X, y, k, return_rankings=False, model=LinearRegression()):
    '''
    This function takes in the X variables as a dataframe, the target variable, number of features to 
    select for modeling (k) with default modle set to Linear Regression. It returns the top k features 
    as well as the rankings for all features.
    '''
    # Use the passed model, LinearRegression by default
    rfe = RFE(model, n_features_to_select=k)
    rfe.fit(X, y)
    features = X.columns[rfe.support_].tolist()
    if return_rankings:
        rankings = pd.Series(dict(zip(X.columns, rfe.ranking_)))
        return features, rankings
    else:
        return features

In [13]:
# call rfe function from explore script
features_to_use, feature_rankings = select_rfe(X_train, y_train, 7, return_rankings=True)

In [14]:
features_to_use

['hours_worked',
 'is_female',
 'is_white',
 'is_hispanic',
 'is_black',
 'is_BIPOC',
 'race_encoded']

In [15]:
features = ['hours_worked',
 'is_female',
 'is_white',
 'is_hispanic',
 'is_black',
 'is_BIPOC',
 'race_encoded']

In [18]:
# 1. Predict mean annual 
sal_pred_mean = y_train.annual_salary.mean()
y_train['sal_pred_mean'] = sal_pred_mean
y_validate['sal_pred_mean'] = sal_pred_mean

# 2. Predict median tax value 
sal_pred_median = y_train.annual_salary.median()
y_train['sal_pred_median'] = sal_pred_median
y_validate['sal_pred_median'] = sal_pred_median

# 3. RMSE of tv_pred_mean
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_mean) ** 0.5
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_mean) ** 0.5

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 5))

# 4. RMSE of tv_pred_median
rmse_train1 = mean_squared_error(y_train.annual_salary, y_train.sal_pred_median) ** .5
rmse_validate1 = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_median) ** .5
print('-----------')
print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train1, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate1, 5))

RMSE using Mean
Train/In-Sample:  25758.77625 
Validate/Out-of-Sample:  25431.1923
-----------
RMSE using Median
Train/In-Sample:  26727.62101 
Validate/Out-of-Sample:  26448.16247


In [19]:
# add the mean baseline to a df to help evaluate all models side by side
metric_df = pd.DataFrame(data=[{
    'model': 'mean_baseline',
    'rmse_outofsample': rmse_validate1,
    'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_mean)}])

## OLS Linear Regression

In [20]:
# create the model object

lm = LinearRegression(fit_intercept=True)

# fit the model to our training data. We must specify the column in y_train, since we have converted it to a dataframe from a series! 

# fit the thing using only the features we selected
lm.fit(X_train[features], y_train.annual_salary)

# predict train

y_train['sal_pred_lm'] = lm.predict(X_train[features])

# evaluate: rmse
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_lm) ** (1/2)

# predict validate
y_validate['sal_pred_lm'] = lm.predict(X_validate[features])

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_lm) ** (1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  24807.084858514223 
Validation/Out-of-Sample:  24522.92556757637


In [21]:
# add model's performance to metric df
metric_df = metric_df.append(
    {
    'model': 'OLS lm',
    'rmse_outofsample': rmse_validate,
        'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_lm)}, ignore_index=True)

In [22]:
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144


## Lasso Lars Model

In [23]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data using only selected features. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series
lars.fit(X_train[features], y_train.annual_salary)

# predict train
y_train['sal_pred_lars'] = lars.predict(X_train[features])

# evaluate: rmse
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_lars) ** 0.5

# predict validate
y_validate['sal_pred_lars'] = lars.predict(X_validate[features])

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_lars) ** 0.5

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for Lasso + Lars
Training/In-Sample:  24855.160340599756 
Validation/Out-of-Sample:  24571.96436221674


In [24]:
# add model to metric df
metric_df = metric_df.append(
    {
    'model': 'LassoLars alpha 1',
    'rmse_outofsample': rmse_validate,
    'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_lars)}, ignore_index=True)

In [25]:
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144
2,LassoLars alpha 1,24571.964362,0.066425


## Tweedie Regressor GLM

In [28]:
# create the model object
glm = TweedieRegressor(power=0, alpha=1)

# fit the model to our training data using only selected features. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series

glm.fit(X_train[features], y_train.annual_salary)

# predict train
y_train['sal_pred_glm'] = glm.predict(X_train[features])

# evaluate: rmse
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_glm) ** 0.5


# predict validate
y_validate['sal_pred_glm'] = glm.predict(X_validate[features])

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_glm) ** 0.5

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for GLM using Tweedie, power=1 & alpha=0
Training/In-Sample:  25184.179628675156 
Validation/Out-of-Sample:  24891.191147839276


In [29]:
# add model to metric df
metric_df = metric_df.append(
    {
    'model': 'Tweedie GLM power 1',
    'rmse_outofsample': rmse_validate,
    'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_glm)}, ignore_index=True)

In [30]:
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144
2,LassoLars alpha 1,24571.964362,0.066425
3,Tweedie GLM power 1,24891.191148,0.042005


## 2nd degree Polynomial Regression Model


In [31]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled features
X_train_degree2 = pf.fit_transform(X_train[features])

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate[features])
X_test_degree2 = pf.transform(X_test[features])


In [32]:
#create the model object

pm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 

# fit the thing

pm.fit(X_train_degree2, y_train.annual_salary)

# predict train

y_train['sal_pred_pm'] = pm.predict(X_train_degree2)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_pm) ** (1/2)

# predict validate
y_validate['sal_pred_pm'] = pm.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_pm) ** (1/2)

print("RMSE for PolynomialRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)


RMSE for PolynomialRegression
Training/In-Sample:  24709.46526854334 
Validation/Out-of-Sample:  24409.465860872413


In [33]:
# add model to metric df
metric_df = metric_df.append(
    {
    'model': 'PolyReg Model',
    'rmse_outofsample': rmse_validate,
    'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_pm)}, ignore_index=True)
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144
2,LassoLars alpha 1,24571.964362,0.066425
3,Tweedie GLM power 1,24891.191148,0.042005
4,PolyReg Model,24409.465861,0.078733


## third degree polynomial

In [34]:
# make the polynomial features to get a new set of features
pf3 = PolynomialFeatures(degree=3)

# fit and transform X_train_scaled features
X_train_degree3 = pf.fit_transform(X_train[features])

# transform X_validate_scaled & X_test_scaled
X_validate_degree3 = pf.transform(X_validate[features])
X_test_degree3 = pf.transform(X_test[features])

In [36]:
#create the model object

pm3 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 

# fit the thing

pm3.fit(X_train_degree3, y_train.annual_salary)

# predict train

y_train['sal_pred_pm3'] = pm3.predict(X_train_degree3)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.annual_salary, y_train.sal_pred_pm3) ** (1/2)

# predict validate
y_validate['sal_pred_pm3'] = pm3.predict(X_validate_degree3)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.annual_salary, y_validate.sal_pred_pm3) ** (1/2)

print("RMSE for PolynomialRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for PolynomialRegression
Training/In-Sample:  24709.46526854334 
Validation/Out-of-Sample:  24409.465860872413


In [37]:
# add model to metric df
metric_df = metric_df.append(
    {
    'model': 'PolyReg Model',
    'rmse_outofsample': rmse_validate,
    'r^2_outofsample': explained_variance_score(y_validate.annual_salary, y_validate.sal_pred_pm3)}, ignore_index=True)
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144
2,LassoLars alpha 1,24571.964362,0.066425
3,Tweedie GLM power 1,24891.191148,0.042005
4,PolyReg Model,24409.465861,0.078733
5,PolyReg Model,24409.465861,0.078733


## Fitting on Test Split

In [42]:
# predict on test
y_test['sal_pred_pm'] = pm.predict(X_test_degree2)

# evaluate: rmse
rmse_test = mean_squared_error(y_test.annual_salary, y_test.sal_pred_pm) ** 0.5
print("RMSE for Model using PolynomialRegression\nOut-of-Sample Performance: ", rmse_test)

RMSE for Model using PolynomialRegression
Out-of-Sample Performance:  24550.200359146158


In [43]:
metric_df = metric_df.append(
    {
    'model': 'Test',
    'rmse_outofsample': rmse_test,
    'r^2_outofsample': explained_variance_score(y_test.annual_salary, y_test.sal_pred_pm)}, ignore_index=True)

In [44]:
metric_df

,model,rmse_outofsample,r^2_outofsample
0,mean_baseline,26448.162472,0.000000
1,OLS lm,24522.925568,0.070144
2,LassoLars alpha 1,24571.964362,0.066425
3,Tweedie GLM power 1,24891.191148,0.042005
4,PolyReg Model,24409.465861,0.078733
5,PolyReg Model,24409.465861,0.078733
6,Test,24550.200359,0.082441
